### **Check if the new structure is not slowing down the code**

In [1]:
using OrdinaryDiffEq, DiffEqDevTools,  Test, BenchmarkTools

import ODEProblemLibrary: prob_ode_linear,
                          prob_ode_2Dlinear


For the previous structure :

In [2]:
@benchmark solve(prob_ode_linear, Tsit5())

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.710 μs … 979.630 μs  ┊ GC (min … max):  0.00% … 99.17%
 Time  (median):     2.250 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.679 μs ±  15.179 μs  ┊ GC (mean ± σ):  14.44% ±  2.94%

               ▅▅█▅                                            
  ▁▃▅▅▅█▆▄▅▃▅▇█████▇▅▄▄▃▃▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁ ▂
  1.71 μs         Histogram: frequency by time        3.95 μs <

 Memory estimate: 4.47 KiB, allocs estimate: 36.

In [3]:
@benchmark solve(prob_ode_2Dlinear, Tsit5())

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.867 μs … 356.233 μs  ┊ GC (min … max): 0.00% … 93.58%
 Time  (median):     6.583 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.681 μs ±  11.279 μs  ┊ GC (mean ± σ):  5.93% ±  4.01%

  ▂▇███▇▅▄▃▃▃▃▃▂▂▁▁▁▂▁                                        ▂
  █████████████████████████▇████▇████████████▇█▇██▇▇▇▇▅▅▅▅▅▅▅ █
  5.87 μs      Histogram: log(frequency) by time      15.1 μs <

 Memory estimate: 14.70 KiB, allocs estimate: 110.

For the new structure :

In [4]:
@benchmark solve(prob_ode_linear, Tsit5_for_relaxation())

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.700 μs … 341.160 μs  ┊ GC (min … max):  0.00% … 98.25%
 Time  (median):     1.980 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.502 μs ±   9.108 μs  ┊ GC (mean ± σ):  11.15% ±  3.09%

  ▂▇█▇▆▅▅▅▅▅▅▅▄▃▂▁▂▁▂▁▁▁▁▂▁▂▂▂▂▂▁▁ ▁                          ▂
  ██████████████████████████████████████▇▇▇▆▆▆▆▆▆▅▆▆▇▇▆▅▆▆▅▂▆ █
  1.7 μs       Histogram: log(frequency) by time      4.79 μs <

 Memory estimate: 4.47 KiB, allocs estimate: 36.

In [5]:
@benchmark solve(prob_ode_2Dlinear, Tsit5_for_relaxation())

BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  5.920 μs … 360.880 μs  ┊ GC (min … max): 0.00% … 95.21%
 Time  (median):     6.380 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.246 μs ±  12.212 μs  ┊ GC (mean ± σ):  6.50% ±  3.80%

   ▄██▇▆▅▅▄▂▂▁▁▁▂▁▃▃▃▃▂▁▂▁                                    ▂
  ████████████████████████▇▇▆██▇▇▇▇▆▄▅▆▆▆▆▅▅▆▄▄▄▄▂▂▅▃▂▄▅▅▆▇▅▅ █
  5.92 μs      Histogram: log(frequency) by time      12.2 μs <

 Memory estimate: 14.70 KiB, allocs estimate: 110.